## Generate Summaries with **Large LLM Longformer Encoder-Decoder (LED)**

The *Longformer Encoder-Decoder (LED)* was recently added as an extension to [Longformer: The Long-Document Transformer](https://arxiv.org/abs/2004.05150) by Iz Beltagy, Matthew E. Peters, Arman Cohan.

In this notebook we will generate summaries based on our previous trained model.

You dont neccessarily need an A100 to run this code.

First, let's check we have a GPU with at least 15GB RAM.

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr  6 00:21:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

We need to upgrade accelerate. Install it on the first session. No need to re-install on the restarted session. Use the pip show to verify you have the version previously installed.

In [ ]:
!pip install accelerate -U --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

Next, we install 🤗Transformers, 🤗Datasets, and `rouge_score`.



Install on the first session. Do no reinstall again after you restart your session as it will upgrade the dill package to 0.3.8., which we do not want.

In [ ]:
%%capture
!pip install datasets==1.2.1 #do not install again, if installed in previous session
!pip install transformers==4.2.0 #do not install again, if installed in previous session
!pip install rouge_score #do not install again, if installed in previous session

Need dill 0.3.4, need to uninstall default dill 0.3.8, otherwise it will have both installations, and the model will fail. After installation wait to restart session AFTER you ALSO reinstall numpy

In [ ]:
!pip uninstall -y dill #need 0.3.4, need to reinstall default dill 0.3.8, otherwise it will have both installations, and model will fail
!pip install dill==0.3.4 #after installation restart session AFTER you ALSO reinstall numpy

Found existing installation: dill 0.3.8
Uninstalling dill-0.3.8:
  Successfully uninstalled dill-0.3.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.16 requires dill>=0.3.8, but you have dill 0.3.4 which is incompatible.


Downgrading numpy version to a previous version that supports np.object. Default numpy installation 1.25.2 causes failure, need to uninstall and install 1.23.5.

In [ ]:
!pip uninstall -y numpy #default numpy installation 1.25.2 causes failure, need to uninstall and install 1.23.5
!pip install numpy==1.23.5 #need to restart session


Found existing installation: numpy 1.25.2
Uninstalling numpy-1.25.2:
  Successfully uninstalled numpy-1.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.


After your install dill and numpy versions needed, you need to restart. After you restart your session, you do not need to install them again. You can verify that you have the expected versions installed by running !pip show dill, or numpy

###Package Version Checks

The next cells are just to check you have the expected versions installed.

In [ ]:
!pip show accelerate #should show 0.28.0 or 0.29.1

Name: accelerate
Version: 0.29.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
!pip show datasets #should show 1.2.1

Name: datasets
Version: 1.2.1
Summary: HuggingFace/Datasets is an open library of NLP datasets.
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: dill, multiprocess, numpy, pandas, pyarrow, requests, tqdm, xxhash
Required-by: 


In [ ]:
!pip show transformers #should show 4.38.2

Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
!pip show rouge_score #should show 0.1.2

Name: rouge-score
Version: 0.1.2
Summary: Pure python implementation of ROUGE-1.5.5.
Home-page: https://github.com/google-research/google-research/tree/master/rouge
Author: Google LLC
Author-email: rouge-opensource@google.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, nltk, numpy, six
Required-by: 


In [ ]:
!pip show dill #need 0.3.4 to work, otherwise you get _stack error.
#if you have not restarted your session it will not work even if it shows 0.3.4


Name: dill
Version: 0.3.4
Summary: serialize all of python
Home-page: https://github.com/uqfoundation/dill
Author: Mike McKerns
Author-email: 
License: 3-clause BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: datasets, multiprocess


In [ ]:
!pip show numpy #should show numpy 1.23.5, otherwise it will not work

Name: numpy
Version: 1.23.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: accelerate, albumentations, altair, arviz, astropy, autograd, blis, bokeh, bqplot, chex, cmdstanpy, contourpy, cufflinks, cupy-cuda12x, cvxpy, datascience, datasets, db-dtypes, dopamine-rl, ecos, flax, folium, geemap, gensim, gym, h5py, holoviews, hyperopt, ibis-framework, imageio, imbalanced-learn, imgaug, jax, jaxlib, librosa, lightgbm, matplotlib, matplotlib-venn, missingno, mizani, ml-dtypes, mlxtend, moviepy, music21, nibabel, numba, numexpr, opencv-contrib-python, opencv-python, opencv-python-headless, opt-einsum, optax, orbax-checkpoint, osqp, pandas, pandas-gbq, pandas-stubs, patsy, plotnine, prophet, pyarrow, pycocotools, pyerfa, pymc, pytensor, python-louvain, PyWavelets, qdldl, qudida, rouge-score, scikit-im

###Begin Loading Data and Defining Functions

In [ ]:
import os
import re
import pandas as pd

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the data

In [ ]:
from tqdm import tqdm


def load_data(file_path):
    chunksize = 5e5 # adjust this value depending on your file's size
    df = pd.DataFrame()
    for chunk in tqdm(pd.read_csv(file_path, chunksize=chunksize)):
        df = pd.concat([df, chunk])
    return df

#  path to where you saved the train data
tr_df = load_data('drive/MyDrive/W266_NLP/Project2/summarization/ms2-train-data.csv') #train data
ts_df = load_data('drive/MyDrive/W266_NLP/Project2/summarization/ms2-test-data.csv') #test data
vl_df = load_data('drive/MyDrive/W266_NLP/Project2/summarization/ms2-val-data.csv')  #validation data


1it [00:08,  8.99s/it]
1it [00:03,  3.03s/it]
1it [00:02,  2.97s/it]


Sanity check the data

In [ ]:
tr_df.head(2)

,Unnamed: 0,review_id,pmid,title,abstract,target,background
0,0,30760312,"['22776744', '25271670', '3493740', '1863023',...",['Improved Cell Survival and Paracrine Capacit...,['Although transplantation of adult bone marro...,Conclusions SC therapy is effective for PAH in...,Background Despite significant progress in dru...
1,1,19588356,"['8532025', '10790348', '17504794', '16793845'...",['A comparison of continuous intravenous epopr...,['BACKGROUND Primary pulmonary hypertension is...,There was a trend for endothelin receptor anta...,BACKGROUND Pulmonary arterial hypertension is ...


Remove columns we will not use. abstract is our input data, and target is our target data.

In [ ]:
# List of columns to drop
columns_to_drop = ['Unnamed: 0', 'review_id', 'pmid', 'title', 'background']

# Drop the unnecessary columns
tr_df = tr_df.drop(columns=columns_to_drop)
ts_df = ts_df.drop(columns=columns_to_drop)
vl_df = vl_df.drop(columns=columns_to_drop)

tr_df.head(2)


,abstract,target
0,['Although transplantation of adult bone marro...,Conclusions SC therapy is effective for PAH in...
1,['BACKGROUND Primary pulmonary hypertension is...,There was a trend for endothelin receptor anta...


In [ ]:
tr_df.shape , vl_df.shape

((14188, 2), (2021, 2))

To check that we are having enough RAM we can run the following command.
If the randomely allocated GPU is too small, the above cells can be run
to crash the notebook hoping to get a better GPU.

In [ ]:
!nvidia-smi

Wed Apr  3 06:48:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Let's start by loading and preprocessing the dataset.



In [ ]:
from datasets import Dataset, load_dataset, load_metric

Next, we download the data from the dataframes.

In [ ]:
# Convert the DataFrame to a Dataset
train_dataset = Dataset.from_pandas(tr_df)
val_dataset = Dataset.from_pandas(vl_df)


##Evaluation

In [ ]:
!nvidia-smi

Thu Apr  4 23:56:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch

from datasets import load_dataset, load_metric
from transformers import LEDTokenizer, LEDForConditionalGeneration

####If there are x in "val_dataset = val_dataset.select(range(x))" it evaluates x entries, resulting in "result.shape (x,3)". If you want to evaluate the entire dataset then you can skip this line, or do "val_dataset.select(range(2021))" which is the entire validation dataset.

In [ ]:
#train_dataset = train_dataset.select(range(10000))
val_dataset = val_dataset.select(range(10))

###Evaluation

####Now load your pretrained model, change load_dir to your saved directory of your fine-tuned model

In [ ]:
load_dir = "/content/drive/MyDrive/W266_NLP/Project2/summarization/e_longformer_8k_plain1/checkpoint-750/"

In [ ]:
# Load the fine-tuned model
model = LEDForConditionalGeneration.from_pretrained(load_dir).to("cuda").half()
# Load the tokenizer
tokenizer = LEDTokenizer.from_pretrained(load_dir)







In [ ]:
max_input_token_length = 8192
max_output_token_length = 196

In [ ]:
def generate_answer(batch):
  inputs_dict = tokenizer(batch["abstract"], padding="max_length", max_length=max_input_token_length, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")
  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
  #batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
  batch["predicted_abstract"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True, max_length=max_output_token_length )

  return batch



In [ ]:
vl_df.shape

(2021, 2)

Rouge1

In [ ]:

result = val_dataset.map(generate_answer, batched=True, batch_size=2)

# load rouge
rouge = load_metric("rouge")

print("Result rouge1:", rouge.compute(predictions=result["predicted_abstract"], references=result["target"], rouge_types=["rouge1"])["rouge1"].mid)


Result rouge1: Score(precision=0.14419244117778088, recall=0.3449759135494807, fmeasure=0.195298387070339)


Because there are 10 in "val_dataset = val_dataset.select(range(10))" it evaluates 10 entries, resulting in "result.shape (10,3)". If you want to evaluate the entier dataset then you can skip that line, or do "val_dataset.select(range(2021))" since there 2021 row entries for validation.

In [ ]:
result.shape

(10, 3)

Rouge2

In [ ]:

result = val_dataset.map(generate_answer, batched=True, batch_size=2)

# load rouge
rouge = load_metric("rouge")

print("Result rouge2:", rouge.compute(predictions=result["predicted_abstract"], references=result["target"], rouge_types=["rouge2"])["rouge2"].mid)


Result rouge2: Score(precision=0.02601644946510874, recall=0.06378369307388822, fmeasure=0.03535102851042486)


Rouge2 and Rouge1

In [ ]:

result = val_dataset.map(generate_answer, batched=True, batch_size=2)

# Load rouge
rouge = load_metric("rouge")

# Compute and print rouge1 and rouge2
rouge1 = rouge.compute(predictions=result["predicted_abstract"], references=result["target"], rouge_types=["rouge1"])["rouge1"].mid
rouge2 = rouge.compute(predictions=result["predicted_abstract"], references=result["target"], rouge_types=["rouge2"])["rouge2"].mid

print("Result rouge1:", rouge1)
print("Result rouge2:", rouge2)


####You can check on certain results, specified by [ i ]

In [ ]:
i = 1

This is what the summary that the model is making:

In [ ]:
result["predicted_abstract"][i]

'Conclusion Glucomannan is an effective dietary adjunct for the treatment of hypercholesterolemic children.\nGlucoman is a safe and efficacious dietary adjunct to the regular diet.\nThe use of GluComannan as an adjunct to regular diet is not associated with a significant reduction in blood lipid concentrations.\nHowever, the use of glucomans as an alternative to regular diets is not likely to be associated with any significant reduction of blood lipid levels.\nIn addition, glu comannan appears to be an effective adjunct to daily diet in children with high-density lipoprotein cholesterol'

This is what the actual summary is (target) from our dataset
:

In [ ]:
result["target"][i]

'The use of glucomannan did not appear to significantly alter any other study endpoints .\nPediatric patients , patients receiving dietary modification , and patients with impaired glucose metabolism did not benefit from glucomannan to the same degree .\nGlucomannan appears to beneficially affect total cholesterol , LDL cholesterol , triglycerides , body weight , and FBG , but not HDL cholesterol or BP'

same as above, but from the dataframe that loaded the csv:

In [ ]:
vl_df.target.iloc[i]

'The use of glucomannan did not appear to significantly alter any other study endpoints .\nPediatric patients , patients receiving dietary modification , and patients with impaired glucose metabolism did not benefit from glucomannan to the same degree .\nGlucomannan appears to beneficially affect total cholesterol , LDL cholesterol , triglycerides , body weight , and FBG , but not HDL cholesterol or BP'